# spark_sql_interactive

In [ ]:
!pip install apache-iceberg

Interactively execute arbitrary SQL queries againts CSV and PARQUET files

In [ ]:
# name of resulting file (default: data_result.csv)
output_result_file = os.environ.get('output_result_file', 'data_result.csv')

# file name for CSV or PARQUET file - must end with .csv or .parquet (default: data.csv)
data_file = os.environ.get('data_file', 'data.csv')

# master url of spark master (default: local mode)
master = os.environ.get('master', "local[*]")

# data_dir temporal data storage for local execution (default: ../../data/)
data_dir = os.environ.get('data_dir', '../../data/')

# sql statement to execute, table name == df, example: select * from df
sql = os.environ.get('sql')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
spark = SparkSession.builder.getOrCreate()

In [ ]:
!ls /home/romeokienzler/gitco/claimed/component-library/transform/spark-sql-interactive/stocator-1.1.5-jar-with-dependencies.jar

In [ ]:
%%bash

#spark-submit --jars /home/romeokienzler/gitco/stocator/target/stocator-1.1.5-jar-with-dependencies.jar --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.14.1 --py-files /home/romeokienzler/gitco/claimed/component-library/transform/spark-sql-interactive/app.py /home/romeokienzler/gitco/claimed/component-library/transform/spark-sql-interactive/app.py
spark-submit --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.14.1\
    --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions \
    --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog \
    --conf spark.sql.catalog.spark_catalog.type=hive \
    --conf spark.sql.catalog.local=org.apache.iceberg.spark.SparkCatalog \
    --conf spark.sql.catalog.local.type=hadoop \
    --jars /home/romeokienzler/gitco/stocator/target/stocator-1.1.5-jar-with-dependencies.jar\
    --conf spark.sql.catalog.local.warehouse=$PWD/warehouse\
    --py-files /home/romeokienzler/gitco/claimed/component-library/transform/spark-sql-interactive/app.py /home/romeokienzler/gitco/claimed/component-library/transform/spark-sql-interactive/app.py

In [ ]:
%%bash

spark-sql --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.14.1\
    --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions \
    --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog \
    --conf spark.sql.catalog.spark_catalog.type=hive \
    --conf spark.sql.catalog.local=org.apache.iceberg.spark.SparkCatalog \
    --conf spark.sql.catalog.local.type=hadoop \
    --conf spark.sql.catalog.local.warehouse=$PWD/warehouse

In [ ]:
%%bash

spark-sql --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.14.1\
    --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions \
    --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog \
    --conf spark.sql.catalog.spark_catalog.type=hive \
    --conf spark.sql.catalog.local=org.apache.iceberg.spark.SparkCatalog \
    --conf spark.sql.catalog.local.type=hadoop \
    --conf spark.sql.catalog.local.warehouse=$PWD/warehouse
    
    conf.set("spark.sql.catalog.nessie.warehouse", "/path/to/warehouse");
conf.set("spark.sql.catalog.nessie.uri", "https://nessie.u7fqdzt4f98.eu-de.codeengine.appdomain.cloud/api/v1")
conf.set("spark.sql.catalog.nessie.ref", "main")
conf.set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
conf.set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")

